# DataStatistics

In [23]:
import sys
sys.path.append('.')
sys.path.append('..')
from glob import glob

import numpy as np
from PIL import Image

from blender_utils import PoseTransformer
from src.config.config import cfg
from src.datasets.LineMod.LineModDB import read_pose

## Setup

In [24]:
linemod_dir = cfg.LINEMOD_DIR
pvnet_linemod_dir = cfg.PVNET_LINEMOD_DIR
object_name = 'ape'

In [25]:
class DataStatistics(object):
    # world_to_camera_pose = np.array([[-1.19209304e-07,   1.00000000e+00,  -2.98023188e-08, 1.19209304e-07],
    #                                  [-8.94069672e-08,   2.22044605e-16,  -1.00000000e+00, 8.94069672e-08],
    #                                  [-1.00000000e+00,  -8.94069672e-08,   1.19209304e-07, 1.00000000e+00]])
    world_to_camera_pose = np.array(
        [[-1.00000024e+00,  -8.74227979e-08,  -5.02429621e-15, 8.74227979e-08],
        [5.02429621e-15,   1.34358856e-07,  -1.00000012e+00, -1.34358856e-07],
        [8.74227979e-08,  -1.00000012e+00,   1.34358856e-07, 1.00000012e+00]])

    def __init__(self,
                              linemod_dir: str,
                              pvnet_linemod_dir: str,
                              object_name):
        self.obj_name = object_name
        self.linemod_dir = cfg.LINEMOD_DIR
        self.pvnet_linemod_dir = cfg.PVNET_LINEMOD_DIR
        self.parent_dir_path = cfg.DATA_DIR

        self.mask_path = os.path.join(self.pvnet_linemod_dir,'{}/mask/*.png'.format(object_name))
        self.dir_path = os.path.join(self.linemod_dir,'{}/data'.format(object_name))

        dataset_pose_dir_path = os.path.join(self.parent_dir_path, 'dataset_poses')
        os.system('mkdir -p {}'.format(dataset_pose_dir_path))
        self.dataset_poses_path = os.path.join(dataset_pose_dir_path, '{}_poses.npy'.format(object_name))
        blender_pose_dir_path = os.path.join(self.parent_dir_path, 'blender_poses')
        os.system('mkdir -p {}'.format(blender_pose_dir_path))
        self.blender_poses_path = os.path.join(blender_pose_dir_path, '{}_poses.npy'.format(object_name))
        os.system('mkdir -p {}'.format(blender_pose_dir_path))

        self.pose_transformer = PoseTransformer(self.obj_name,
                                                                                               self.pvnet_linemod_dir,
                                                                                               self.linemod_dir)

    def get_proper_crop_size(self):
        """
        mask path のディレクトリ内に存在するすべての .png 画像の大きさを求め，その中で 最大の幅と高さ および 最小の幅と高さ を返す関数
        """
        mask_paths = glob(self.mask_path)
        widths = []
        heights = []

        for mask_path in mask_paths:
            mask = Image.open(mask_path).convert('1')
            mask = np.array(mask).astype(np.int32)
            row_col = np.argwhere(mask == 1)
            min_row, max_row = np.min(row_col[:, 0]), np.max(row_col[:, 0])
            min_col, max_col = np.min(row_col[:, 1]), np.max(row_col[:, 1])
            width = max_col - min_col
            height = max_row - min_row
            widths.append(width)
            heights.append(height)

        widths = np.array(widths)
        heights = np.array(heights)
        print('min width: {}, max width: {}'.format(np.min(widths), np.max(widths)))
        print('min height: {}, max height: {}'.format(np.min(heights), np.max(heights)))

    def get_quat_translation(self, object_to_camera_pose):

        object_to_camera_pose = np.append(object_to_camera_pose, [[0, 0, 0, 1]], axis=0)
        world_to_camera_pose = np.append(self.world_to_camera_pose, [[0, 0, 0, 1]], axis=0)
        object_to_world_pose = np.dot(np.linalg.inv(world_to_camera_pose), object_to_camera_pose)
        quat = mat2quat(object_to_world_pose[:3, :3])
        translation = object_to_world_pose[:3, 3]
        return quat, translation

    def get_dataset_poses(self):
        if os.path.exists(self.dataset_poses_path):
            poses = np.load(self.dataset_poses_path)
            return poses[:, :3], poses[:, 3:]

        eulers = []
        translations = []
        train_set = np.loadtxt(os.path.join(self.pvnet_linemod_dir, '{}/training_range.txt'.format(self.obj_name)),np.int32)
        for idx in train_set:
            rot_path = os.path.join(self.dir_path, 'rot{}.rot'.format(idx))
            tra_path = os.path.join(self.dir_path, 'tra{}.tra'.format(idx))
            pose = read_pose(rot_path, tra_path)
            euler = self.pose_transformer.orig_pose_to_blender_euler(pose)
            eulers.append(euler)
            translations.append(pose[:, 3])

        eulers = np.array(eulers)
        translations = np.array(translations)
        np.save(self.dataset_poses_path, np.concatenate([eulers, translations], axis=-1))

        return eulers, translations

    def sample_sphere(self, num_samples):
        """ sample angles from the sphere
        reference: https://zhuanlan.zhihu.com/p/25988652?group_id=828963677192491008
        """
        flat_objects = ['037_scissors', '051_large_clamp', '052_extra_large_clamp']
        if self.object_name in flat_objects:
            begin_elevation = 30
        else:
            begin_elevation = 0
        ratio = (begin_elevation + 90) / 180
        num_points = int(num_samples // (1 - ratio))
        phi = (np.sqrt(5) - 1.0) / 2.
        azimuths = []
        elevations = []
        for n in range(num_points - num_samples, num_points):
            z = 2. * n / num_points - 1.
            azimuths.append(np.rad2deg(2 * np.pi * n * phi % (2 * np.pi)))
            elevations.append(np.rad2deg(np.arcsin(z)))
        return np.array(azimuths), np.array(elevations)

    def sample_poses(self):
        eulers, translations = self.get_dataset_poses()
        num_samples = cfg.NUM_SYN
        azimuths, elevations = self.sample_sphere(num_samples)
        euler_sampler = stats.gaussian_kde(eulers.T)
        eulers = euler_sampler.resample(num_samples).T
        eulers[:, 0] = azimuths
        eulers[:, 1] = elevations
        translation_sampler = stats.gaussian_kde(translations.T)
        translations = translation_sampler.resample(num_samples).T
        np.save(self.blender_poses_path, np.concatenate([eulers, translations], axis=-1))

## インスタンス生成

In [26]:
ds = DataStatistics(linemod_dir = linemod_dir,
                                       pvnet_linemod_dir = pvnet_linemod_dir,
                                       object_name = object_name)

## get_proper_crop_size Test

In [27]:
ds.get_proper_crop_size()

min width: 34, max width: 67
min height: 35, max height: 88


## get_dataset_poses

In [28]:
eulers, translations = ds.get_dataset_poses()

AttributeError: 'PoseTransformer' object has no attribute 'orig_pose_to_blender_euler'